In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from SALib import ProblemSpec
from seapopym.configuration.no_transport.parameter import ForcingParameters, ForcingUnit, KernelParameters
from dask.distributed import Client
import dask
import plotly.express as px
from plotly.subplots import make_subplots
from seapopym_optimization import wrapper

Load forcing.


In [2]:
STATION = "Bats"

In [3]:
time_start, time_end = "1998-01-01", "2022-01-01"

data = xr.open_dataset(f"../1_data_processing/1_1_Forcing/products/{STATION}_cmems.zarr", engine="zarr")
data["time"].attrs = {"axis": "T"}
data = data.sel(time=slice(time_start, time_end))
_ = data.load()

cafe_npp = xr.open_dataset(f"../1_data_processing/1_1_Forcing/products/{STATION}_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.time.attrs["axis"] = "T"
_ = cafe_npp.load()

In [4]:
data

<xarray.Dataset> Size: 2MB
Dimensions:              (latitude: 1, longitude: 1, time: 8767, depth: 3)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 210kB 20....
    U                    (latitude, longitude, time, depth) float64 210kB 0.0...
    V                    (latitude, longitude, time, depth) float64 210kB -0....
    mnkc_epi             (latitude, longitude, time) float64 70kB 0.7326 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 70kB 0.6836 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 70kB 1.118 ... 1...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 70kB 0.579 ... 1...
    mnkc_umeso           (latitude, longitude, time) float64 70kB 0.3336 ... ...
    npp                  (latitude, longitude, time) float64 70kB 450.0 ... 2...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 210kB 100...
    zeu                  (latitude, longitude, time) float64 70kB 66.78 ... 7...
    zooc                 (latitude, longitude, time) float64 70kB 0.3742 ... ...

In [5]:
cafe_npp

<xarray.Dataset> Size: 140kB
Dimensions:    (time: 8767, latitude: 1, longitude: 1)
Coordinates:
  * latitude   (latitude) float64 8B 31.6
  * longitude  (longitude) float64 8B -64.2
  * time       (time) datetime64[ns] 70kB 1998-01-01 1998-01-02 ... 2022-01-01
Data variables:
    CAFE       (time, latitude, longitude) float64 70kB 262.5 257.1 ... 228.4
Attributes:
    Conventions:  CF-1.8
    History:      2023-01-02 08:55:09.514113 Python
    Keywords:     Primary Production
    PI Contact:   tryankeogh@csir.co.za
    PI Name:      Thomas Ryan-Keogh

In [6]:
forcing_parameter_initial = ForcingParameters(
    temperature=ForcingUnit.from_dataset(forcing=data, name="T", resolution=0.08333, timestep=1),
    primary_production=ForcingUnit.from_dataset(cafe_npp, name="CAFE", resolution=0.08333, timestep=1),
)

2024-11-20 16:50:23,078 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Generate initial conditions.


In [7]:
fg_parameters = wrapper.FunctionalGroupGeneratorNoTransport(
    [[10.38, -0.11, 150, 0.15, 1, 1, 0.1668]],
)
model = wrapper.model_generator_no_transport(
    fg_parameters=fg_parameters,
    forcing_parameters=forcing_parameter_initial,
    kernel_parameters=KernelParameters(compute_initial_conditions=True),
)
model.run()
initial_conditions = model.export_initial_conditions()

2024-11-20 16:50:23,095 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-20 16:50:23,099 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-20 16:50:23,151 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-11-20 16:50:23,189 :: Seapodym ::  DEBUG ::
|	Direct computation for _average_temperature.

2024-11-20 16:50:23,203 :: Seapodym ::  DEBUG ::
|	Direct computation for _apply_coefficient_to_primary_production_helper.

2024-11-20 16:50:23,210 :: Seapodym ::  DEBUG ::
|	Direct computation for _min_temperature_by_cohort_helper.

2024-11-20 16:50:23,212 :: Seapodym ::  DEBUG ::
|	Direct computation for _mask_temperature_helper.

2024-11-20 16:50:23,218 :: Seapodym ::  DEBUG ::
|	Direct computation for _cell_area_helper.

2024-11-20 16:50:23,240 :: Seapodym ::  DEBUG ::
|	Direct computation for _mortality_field_helper.

2024-11-20 16:50:23,245 :: Seapodym ::  DEBUG ::
|	Direct computation for _productio

Model generation used in sensibility analysis.


In [8]:
FORCING_PARAMETERS = ForcingParameters(
    temperature=ForcingUnit.from_dataset(data, name="T", resolution=0.08333, timestep=1),
    primary_production=ForcingUnit.from_dataset(cafe_npp, name="CAFE", resolution=0.08333, timestep=1),
    initial_condition_biomass=ForcingUnit.from_dataset(
        initial_conditions, name="initial_condition_biomass", resolution=0.08333, timestep=1
    ),
    initial_condition_production=ForcingUnit.from_dataset(
        initial_conditions, name="initial_condition_production", resolution=0.08333, timestep=1
    ),
)


def wrapper_model_generator_no_transport(fg_parameters):
    fg_parameters = wrapper.FunctionalGroupGeneratorNoTransport(np.array([fg_parameters]))
    return wrapper.model_generator_no_transport(
        fg_parameters=fg_parameters,
        forcing_parameters=FORCING_PARAMETERS,
    )

2024-11-20 16:50:26,445 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup sensitivity analysis structure.


In [9]:
sp = ProblemSpec(
    {
        "names": [
            "energy_transfert",
            "tr_max",
            "tr_rate",
            "inv_lambda_max",
            "inv_lambda_rate",
        ],
        "groups": None,
        "bounds": [
            [0, 1],
            [0, 50],
            [-1, 0],
            [0, 1000],
            [0, 1],
        ],
        "outputs": ["mean", "variance"],
    }
)

Create the cost function.


In [10]:
@dask.delayed
def cost_function(x: np.ndarray):
    energy_transfert, tr_max, tr_rate, inv_lambda_max, inv_lambda_rate = x.T
    fg_parameters = [tr_max, tr_rate, inv_lambda_max, inv_lambda_rate, 1, 1, energy_transfert]
    model = wrapper_model_generator_no_transport(fg_parameters)

    model.run()
    biomass_forcing = model.export_biomass()

    mean = float(biomass_forcing.mean())
    variance = float(biomass_forcing.var())

    return mean, variance

# SOBOL SENSITIVITY ANALYSIS


In [11]:
client = Client()
client

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59236 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:59236/status,
Dashboard: http://127.0.0.1:59236/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59237,Workers: 4
Dashboard: http://127.0.0.1:59236/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:59254,Total threads: 2
Dashboard: http://127.0.0.1:59255/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:59240,


In [12]:
SAMPLE_NUMBER = 20

In [13]:
param_values = sp.sample_sobol(SAMPLE_NUMBER)
res = [cost_function(param) for param in param_values.samples]
res = client.compute(res)
res = client.gather(res)
sp.set_results(np.asarray(res))

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/scipy/stats/_qmc.py:958: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
2024-11-20 16:50:30,324 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-20 16:50:30,326 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-20 16:50:30,327 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-20 16:50:30,330 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-20 16:50:30,330 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-20 16:50:30,331 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-20 16:50:30,332 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-20 16:50:30,333 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan

Samples:
	5 parameters: ['energy_transfert', 'tr_max', 'tr_rate', 'inv_lambda_max', 'inv_lambda_rate']
	240 samples
Outputs:
	2 outputs: ['mean', 'variance']
	240 evaluations


In [14]:
sp.analyze_sobol()

(mean_total_Si, mean_first_Si, mean_second_Si), (var_total_Si, var_first_Si, var_second_Si) = sp.to_df()

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


In [15]:
def plot_sobol(total_si, first_si, second_si, title):
    fig_total = px.bar(total_si, x=total_si.index, y="ST", error_y="ST_conf")
    fig_first = px.bar(first_si, x=first_si.index, y="S1", error_y="S1_conf")
    fig_second = px.bar(second_si, x=second_si.index.map(str), y="S2", error_y="S2_conf")

    figure_mean = (
        make_subplots(
            rows=3,
            cols=1,
            subplot_titles=("Total Sobol indice", "First Sobol indice", "Second Sobol indice"),
            specs=[[{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]],
            # reduce space between the plots
            vertical_spacing=0.05,
        )
        .add_trace(fig_total.data[0], row=1, col=1)
        .add_trace(fig_first.data[0], row=2, col=1)
        .add_trace(fig_second.data[0], row=3, col=1)
    )
    figure_mean.update_layout(height=1200, width=800, title_text=title)
    return figure_mean

In [16]:
plot_sobol(mean_total_Si, mean_first_Si, mean_second_Si, f"Sobol indices for mean biomass in {STATION}")

In [17]:
plot_sobol(var_total_Si, var_first_Si, var_second_Si, f"Sobol indices for variance biomass in {STATION}")